In [46]:
import os
import gym
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('data\RVNBTC-5m-data.csv').iloc[:10]

In [10]:
df.set_index('Timestamp', inplace=True)

In [16]:
df = df.iloc[:, 1:4]

In [17]:
df_2 = pd.read_csv('data\RVNUSDT-5m-data.csv').iloc[:10]
df_2.set_index('Timestamp', inplace=True)
df_2 = df_2.iloc[:, 1:4]

<>:1: DeprecationWarning: invalid escape sequence \R


In [41]:
data_list = os.listdir('data')
high_df = pd.DataFrame()
low_df = pd.DataFrame()
close_df = pd.DataFrame()

for i in data_list:
    temp_df = pd.read_csv(f'data/{i}') # assume all the data have the same length
    high_df = pd.concat([high_df, temp_df['High']], axis=1, ignore_index=True)
    low_df = pd.concat([low_df, temp_df['Low']], axis=1, ignore_index=True)
    close_df = pd.concat([close_df, temp_df['Close']], axis=1, ignore_index=True)

In [45]:
close = close_df.to_numpy()
high = high_df.to_numpy()
low = low_df.to_numpy()

In [54]:
final_df = np.stack((high, low, close))

In [62]:
high[-1]

array([2.e-06,    nan])

In [64]:
final_df.shape

(3, 337244, 2)

In [23]:
high_df = pd.DataFrame()

In [27]:
pd.concat([high_df, df['High']], axis=1)

,High
2018-10-12 04:00:00,0.000003
2018-10-12 04:05:00,0.000003
2018-10-12 04:10:00,0.000003
2018-10-12 04:15:00,0.000003
2018-10-12 04:20:00,0.000003
2018-10-12 04:25:00,0.000003
2018-10-12 04:30:00,0.000003
2018-10-12 04:35:00,0.000003
2018-10-12 04:40:00,0.000003
2018-10-12 04:45:00,0.000003


In [25]:
pd.concat([df['High'], df_2['High']], axis=1)

,High,High
Timestamp,,
2018-10-12 04:00:00,0.000003,NaN
2018-10-12 04:05:00,0.000003,NaN
2018-10-12 04:10:00,0.000003,NaN
2018-10-12 04:15:00,0.000003,NaN
2018-10-12 04:20:00,0.000003,NaN
2018-10-12 04:25:00,0.000003,NaN
2018-10-12 04:30:00,0.000003,NaN
2018-10-12 04:35:00,0.000003,NaN
2018-10-12 04:40:00,0.000003,NaN
